# Object Detection (Lecture 13)

# 第一部分：2D 物体检测 (Object Detection)

物体检测的目标不仅是识别图像中有什么物体（分类），还要精确定位出它们的位置，通常是用一个**边界框 (Bounding Box)** 把它框出来。

## 单一物体检测

*   **任务**: 定位 (Localization) + 分类 (Classification)。
*   **输出**: 一个边界框和该框内物体的类别。
*   **参数化**: 边界框通常用4个值表示，如 `(x, y, w, h)`，即中心点坐标、宽度和高度。
*   **方法**: 将定位和分类两者结合，形成一个**多任务损失 (Multitask Loss)** 函数，同时优化。

### 回归损失函数

在预测边界框的坐标时，需要一个损失函数来衡量预测框与真实框的差距。
*   **L1 Loss**: $L_1 = \sum_i |\Delta_i|$。对异常值（大的误差）不敏感，比较鲁棒，但在接近收敛时（x=0附近）不够稳定。
*   **L2 Loss**: $L_2 = \sum_i \Delta_i^2$。对误差很敏感，收敛性好，但容易受异常值影响。
*   **Smooth L1 Loss**: Fast R-CNN 提出的一种结合了 L1 和 L2 优点的损失函数。在误差较小时表现像 L2，在误差较大时表现像 L1，做到了既稳定又鲁棒。

    $
    \text{smooth}_{L_1}(x) = 
    \begin{cases} 
    0.5x^2 & \text{if } |x| < 1 \\
    |x| - 0.5 & \text{otherwise}
    \end{cases}
    $

## 多物体检测

最大的挑战是**输出数量不固定**。有的图只有1个物体，有的图有几十个。早期的方法，如**滑窗法 (Sliding-Window)**，虽然直观，但需要对图像上成千上万个不同位置、不同大小的窗口都跑一遍分类器，计算量巨大，效率极低。

为了解决滑窗法的低效问题，研究者们提出了基于“区域提议 (Region Proposal)”的 R-CNN 系列算法

###   **R-CNN (Regions with CNN features)**:
1.  **区域提议**: 先用传统算法（如 Selective Search）在图像上生成约2000个可能包含物体的候选区域 (Region of Interest, RoI)。
2.  **特征提取**: 将每个候选区域强制缩放到固定大小 (如 224x224)，然后**独立地**送入一个卷积神经网络 (CNN) 提取特征。
3.  **分类与回归**: 用支持向量机 (SVM) 对提取的特征进行分类，并用一个线性回归器微调边界框的位置。
*   **缺点**:
    *   **速度极慢**: 对2000个区域重复进行CNN前向传播，计算冗余极大。
    *   **训练复杂**: 训练分为多个独立阶段（提议、CNN、SVM、回归器），不是端到端的。

###   **Fast R-CNN**:
核心思想是**共享计算**。
1.  **共享特征提取**: 不再对每个区域单独计算，而是**将整张图输入CNN一次**，得到一个全局的特征图 (feature map)。
2.  **RoI Pooling**: 将原始图像上的候选区域映射到特征图上，然后通过一个叫做 **RoI Pooling** 的层，将不同大小的特征区域转换成固定大小的特征向量。
3.  **统一预测**: 将固定大小的特征向量送入全连接层，**同时**完成分类和边界框回归。
*   **优点**: 速度比 R-CNN 快得多（训练快10倍，测试快近200倍），并且是端到端训练（除了区域提议）。
*   **瓶颈**: 区域提议步骤（Selective Search）成了新的速度瓶颈。

#### **投影**

1.  **背景**: 在 Fast R-CNN 中，我们不再像老方法 R-CNN 那样，把成千上万个候选框（proposal）分别送入神经网络。而是先把**整张大图**（左边 640x480 的猫图）输入到一个CNN网络里，得到一个浓缩后的**特征图 (feature map)**（右边 512x20x15 的立方体）。这个特征图虽然变小了（从 640x480 缩小到 20x15），但它包含了原图的高级语义信息。

2.  **操作**: “投影”就是要把原始图像上的那个绿色候选框（框住了猫），按照同样的缩放比例，映射到这个小小的特征图上。缩放比例是由CNN的结构决定的。

#### **“吸附”到网格单元**

RoI Pooling **取整**。它会把这些小数坐标强制“吸附”到最近的整数网格线上。

### Faster R-CNN

在Fast R-CNN中区域提议步骤（Selective Search）成了新的速度瓶颈。所以Faster就选择让CNN来进行 **找框**。

#### **第一步：铺设 Anchor Boxes**

在特征图的**每一个像素点**上，RPN 都预先放置了 $k$ 个（通常是9个）形状各异的虚拟框。这些框就叫**锚框 (Anchor)**。
*   有的框是方的，有的框是扁的，有的框是瘦高的。
*   有的框很大，有的框很小。

如果特征图大小是 $20 \times 15$，每个点放 9 个锚框，那总共就有 $20 \times 15 \times 9 = 2700$ 个锚框。这 2700 个框铺满了整张图。

#### **第二步：RPN 的判断 (分类 + 回归)**

RPN 拿着放大镜扫描这 2700 个锚框，对**每一个锚框**做两个判断：

1.  **有没有东西？(Objectness Score)**
    *   RPN 会给每个锚框打分：“这个框里像是有个物体（前景）” 还是 “这个框里全是背景（墙壁、草地）”。
    *   如果是背景，这个锚框就被扔掉了。

2.  **位置准不准？(Box Regression)**
    *   如果 RPN 觉得这个锚框里有物体，它会接着看：“这个锚框虽然套住了物体，但好像偏了一点，或者太大了。”
    *   于是它会输出 4 个修正值 `(dx, dy, dw, dh)`，意思是：“把这个锚框往左移一点，再变窄一点，就能完美套住物体了。”

#### **第三步：交接工作**

经过 RPN 的处理，把框变成了约 300 个**高质量的候选框 (Proposals)**。RPN 把这 300 个框交给Fast R-CNN 的后续部分。

#### 非极大值抑制 (Non-Maximal Suppression, NMS)**

检测器通常会针对同一个物体输出多个重叠的边界框，NMS 是一种后处理算法，用于剔除多余的框，只保留置信度最高且重叠度不高的那个。

NMS 的详细流程：

**准备工作:**

*   **B**: 一个包含**所有**原始预测框的列表。每个框都有自己的坐标和置信度分数。
*   **$\tau$ (IoU Threshold)**: 一个我们自己设定的**重叠度门槛**（比如 0.5）。如果两个框的重叠度（IoU）超过这个值，我们就认为它们框的是同一个物体。
*   **D**: 一个空的列表，用来存放我们最终筛选出来的结果。

**循环：**

1.  **选出最自信的王**：
    *   **操作**: 从列表 **B** 中，找到那个**置信度分数最高**的框。

2.  **“王”直接晋级**：
    *   **操作**: 将这个框从列表 **B** 中**移除**，并把它**放入**最终结果列表 **D** 中。这个框我们肯定要了。

3.  **“王”干掉身边的小弟**：
    *   **操作**: 遍历列表 **B** 中**剩下**的所有框。计算每一个框与 `box_king` 的 **IoU (交并比)**。
    *   **规则**: 如果某个框与 `box_king` 的 IoU **大于**我们设定的门槛 $\tau$（比如 > 0.5），说明它和“王”的重叠度太高了，只是个重复的检测。那就把它从列表 **B** 中**删除**。

从B中剩下部分选出最高的，循环上面步骤，直到原始列表 **B** 被清空。列表 **D** 中剩下的，就是经过 NMS 筛选后的最终检测结果——每个物体只保留一个最自信的边界框。

